# Gaussian Process et fluctuations de Levy sur un phénomène saisonnier propose par MistralAI

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.tsa.seasonal import STL
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C

In [ ]:
# Exemple de données synthétiques
np.random.seed(0)
n = 1000
dates = pd.date_range(start='1/1/2020', periods=n, freq='D')
data = np.sin(2 * np.pi * dates.dayofyear / 365.25) + np.random.randn(n) * 0.5

# Créer un DataFrame avec un index de type DatetimeIndex
df = pd.DataFrame(data, index=dates, columns=['value'])

# Décomposition saisonnière
stl = STL(df['value'], seasonal=13)
res = stl.fit()

# Extraire les composantes
trend = res.trend
seasonal = res.seasonal
residual = res.resid

# Modélisation des résidus avec un processus gaussien
kernel = C(1.0) * RBF(length_scale=1.0)
gp = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=10)

# Ajuster le modèle aux résidus
X = np.arange(len(residual)).reshape(-1, 1)
gp.fit(X, residual)

# Prédire avec le modèle
X_pred = np.arange(len(residual), len(residual) + 30).reshape(-1, 1)
y_pred, sigma = gp.predict(X_pred, return_std=True)

# Visualisation
plt.figure(figsize=(12, 8))
plt.subplot(311)
plt.plot(df.index, df['value'], label='Données originales')
plt.plot(df.index, trend, label='Tendance', color='red')
plt.legend()

plt.subplot(312)
plt.plot(df.index, seasonal, label='Saisonnière', color='green')
plt.legend()

plt.subplot(313)
plt.plot(df.index, residual, label='Résidus', color='blue')
plt.plot(pd.date_range(start=df.index[-1], periods=31, freq='D')[1:], y_pred, label='Prédiction GP', color='orange')
plt.fill_between(pd.date_range(start=df.index[-1], periods=31, freq='D')[1:],
                 y_pred - sigma,
                 y_pred + sigma,
                 alpha=0.2, color='orange', label='Incertitude GP')
plt.legend()

plt.tight_layout()
plt.show()


## Processus de Levy

Pour ajuster un processus de Lévy à vos données, vous pouvez utiliser des bibliothèques spécialisées qui implémentent ces processus. Cependant, l'ajustement d'un processus de Lévy peut être complexe, car il implique souvent l'estimation de paramètres pour des distributions à queues lourdes, comme les distributions alpha-stables.

Voici une approche générale pour ajuster un processus de Lévy à vos résidus :

Étapes pour ajuster un processus de Lévy :
Prétraitement des données :

Assurez-vous que les résidus sont correctement extraits après la décomposition saisonnière.
Estimation des paramètres du processus de Lévy :

Utilisez une bibliothèque comme PyLevy ou scipy pour estimer les paramètres d'une distribution alpha-stable, qui est souvent utilisée pour modéliser les sauts dans un processus de Lévy.
Simulation et ajustement :

Simulez un processus de Lévy avec les paramètres estimés et comparez-le aux résidus observés.
Ajustez les paramètres pour minimiser l'écart entre les résidus observés et simulés.
Exemple de code :
Voici un exemple simplifié utilisant scipy pour estimer les paramètres d'une distribution alpha-stable :



### Explications :
- levy_stable.fit : Utilisé pour estimer les paramètres d'une distribution alpha-stable à partir des résidus.
- Simulation : Un processus de Lévy est simulé avec les paramètres estimés pour comparaison avec les résidus observés.
- Processus gaussien : Utilisé pour modéliser les variations continues des résidus.
Ce code illustre comment ajuster un processus de Lévy aux résidus après avoir retiré les composantes saisonnières et de tendance. Si vous avez besoin d'aide supplémentaire ou d'explications, n'hésitez pas à demander !

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import levy_stable
from statsmodels.tsa.seasonal import STL
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C

# Exemple de données synthétiques
np.random.seed(0)
n = 1000
dates = pd.date_range(start='1/1/2020', periods=n, freq='D')
data = np.sin(2 * np.pi * dates.dayofyear / 365.25) + np.random.randn(n) * 0.5

# Créer un DataFrame avec un index de type DatetimeIndex
df = pd.DataFrame(data, index=dates, columns=['value'])

# Décomposition saisonnière
stl = STL(df['value'], seasonal=13)
res = stl.fit()

# Extraire les composantes
trend = res.trend
seasonal = res.seasonal
residual = res.resid

# Ajustement d'une distribution alpha-stable aux résidus
alpha, beta, gamma, delta = levy_stable.fit(residual)

print(alpha,beta,gamma,delta)

# Simuler un processus de Lévy avec les paramètres estimés
simulated_levy = levy_stable.rvs(alpha, beta, loc=delta, scale=gamma, size=n)

# Modélisation des résidus avec un processus gaussien
kernel = C(1.0) * RBF(length_scale=1.0)
gp = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=10)

# Ajuster le modèle aux résidus
X = np.arange(len(residual)).reshape(-1, 1)
gp.fit(X, residual)

# Prédire avec le modèle
X_pred = np.arange(len(residual), len(residual) + 30).reshape(-1, 1)
y_pred, sigma = gp.predict(X_pred, return_std=True)

# Visualisation
plt.figure(figsize=(12, 8))
plt.subplot(311)
plt.plot(df.index, df['value'], label='Données originales')
plt.plot(df.index, trend, label='Tendance', color='red')
plt.legend()

plt.subplot(312)
plt.plot(df.index, seasonal, label='Saisonnière', color='green')
plt.legend()

plt.subplot(313)
plt.plot(df.index, residual, label='Résidus', color='blue')
plt.plot(df.index, simulated_levy, label='Processus de Lévy simulé', color='purple', alpha=0.6)
plt.plot(pd.date_range(start=df.index[-1], periods=31, freq='D')[1:], y_pred, label='Prédiction GP', color='orange')
plt.fill_between(pd.date_range(start=df.index[-1], periods=31, freq='D')[1:],
                 y_pred - sigma,
                 y_pred + sigma,
                 alpha=0.2, color='orange', label='Incertitude GP')
plt.legend()

plt.tight_layout()
plt.show()


In [ ]:
plt.hist(simulated_levy,bins=100)

Explications :
gaussian_kde : Utilisé pour estimer la densité de probabilité des données simulées. Cela permet de tracer une courbe lisse qui représente la distribution sous-jacente des données.
density=True : Assure que l'histogramme est normalisé pour représenter une densité de probabilité.
alpha=0.6 : Rend l'histogramme semi-transparent pour mieux voir la courbe de densité.
Ce code génère un histogramme des données simulées avec une courbe de densité de probabilité superposée, ce qui permet de visualiser la distribution des données simulées. Si vous avez d'autres questions ou besoin d'aide supplémentaire, n'hésitez pas à demander !

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import levy_stable, gaussian_kde
from statsmodels.tsa.seasonal import STL
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C

# Exemple de données synthétiques
np.random.seed(0)
n = 1000
dates = pd.date_range(start='1/1/2020', periods=n, freq='D')
data = np.sin(2 * np.pi * dates.dayofyear / 365.25) + np.random.randn(n) * 0.5

# Créer un DataFrame avec un index de type DatetimeIndex
df = pd.DataFrame(data, index=dates, columns=['value'])

# Décomposition saisonnière
stl = STL(df['value'], seasonal=13)
res = stl.fit()

# Extraire les composantes
trend = res.trend
seasonal = res.seasonal
residual = res.resid

# Ajustement d'une distribution alpha-stable aux résidus
alpha, beta, gamma, delta = levy_stable.fit(residual)

# Simuler un processus de Lévy avec les paramètres estimés
simulated_levy = levy_stable.rvs(alpha, beta, loc=delta, scale=gamma, size=n)

# Tracer l'histogramme et la densité de probabilité
plt.figure(figsize=(8, 6))

# Histogramme des données simulées
plt.hist(simulated_levy, bins=50, density=True, alpha=0.6, color='g', label='Histogramme des données simulées')

# Estimation de la densité de probabilité
kde = gaussian_kde(simulated_levy)
x = np.linspace(min(simulated_levy), max(simulated_levy), 1000)
plt.plot(x, kde(x), label='Densité de probabilité estimée', color='blue')

# Ajouter des labels et une légende
plt.xlabel('Valeur')
plt.ylabel('Densité')
plt.title('Histogramme et densité de probabilité des données simulées de Lévy')
plt.legend()

plt.show()
